In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Yolo_training')

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git


Cloning into 'yolov5'...
remote: Enumerating objects: 17483, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 17483 (delta 77), reused 29 (delta 29), pack-reused 17378 (from 4)
Receiving objects: 100% (17483/17483), 16.35 MiB | 9.44 MiB/s, done.
Resolving deltas: 100% (11988/11988), done.


In [ ]:
os.chdir('yolov5')

In [ ]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import os
import random
import shutil

# Set random seed for reproducibility
random.seed(42)

# Base directory where images are stored
base_dir = '/content/drive/MyDrive/Colab Notebooks/Yolo_training/yolov5/data_images'
all_files = [f for f in os.listdir(base_dir) if f.endswith('.jpg') or f.endswith('.png')]

# Shuffle the list of images
random.shuffle(all_files)

# Calculate the split index
split_index = int(0.8 * len(all_files))

# Create train and val folders if they don't exist
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move files
for i, file in enumerate(all_files):
    # Determine source and destination
    src_img_path = os.path.join(base_dir, file)
    dst_folder = train_dir if i < split_index else test_dir

    # Move image
    shutil.move(src_img_path, os.path.join(dst_folder, file))

    # Also move corresponding label file if it exists
    label_file = os.path.splitext(file)[0] + '.txt'
    label_path = os.path.join(base_dir, label_file)
    if os.path.exists(label_path):
        shutil.move(label_path, os.path.join(dst_folder, label_file))

print("✅ Dataset split completed successfully.")


✅ Dataset split completed successfully.


In [ ]:
import os
import shutil

# Base directories
base_dir = '/content/drive/MyDrive/Colab Notebooks/Yolo_training/yolov5/data_images'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Function to move XMLs based on .jpg files
def move_xmls(image_dir):
    for file in os.listdir(image_dir):
        if file.lower().endswith(('.jpg', '.png')):
            xml_filename = os.path.splitext(file)[0] + '.xml'
            src_xml_path = os.path.join(base_dir, xml_filename)
            dst_xml_path = os.path.join(image_dir, xml_filename)
            if os.path.exists(src_xml_path):
                shutil.move(src_xml_path, dst_xml_path)

# Move XML files to corresponding folders
move_xmls(train_dir)
move_xmls(test_dir)

print("✅ Moved corresponding XML files to train/ and val/ folders.")


✅ Moved corresponding XML files to train/ and val/ folders.


In [ ]:
import os
import xml.etree.ElementTree as ET

# Set paths
base_dir = '/content/drive/MyDrive/Colab Notebooks/Yolo_training/yolov5/data_images'
sets = ['train', 'test']
classes = ['face']

def convert_box(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2]) / 2.0 - 1
    y = (box[1] + box[3]) / 2.0 - 1
    w = box[2] - box[0]
    h = box[3] - box[1]
    return (x * dw, y * dh, w * dw, h * dh)

for image_set in sets:
    folder = os.path.join(base_dir, image_set)
    for filename in os.listdir(folder):
        if not filename.endswith('.xml'):
            continue
        xml_path = os.path.join(folder, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        size = root.find('size')
        if size is None:
            continue
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        label_file = os.path.splitext(xml_path)[0] + ".txt"
        with open(label_file, 'w') as out_file:
            for obj in root.iter('object'):
                cls = obj.find('name').text
                if cls not in classes:
                    continue
                cls_id = classes.index(cls)
                xmlbox = obj.find('bndbox')
                b = (float(xmlbox.find('xmin').text), float(xmlbox.find('ymin').text),
                     float(xmlbox.find('xmax').text), float(xmlbox.find('ymax').text))
                bb = convert_box((w, h), b)
                out_file.write(f"{cls_id} {' '.join([str(a) for a in bb])}\n")

print("✅ XML files converted to YOLO .txt format.")


✅ XML files converted to YOLO .txt format.


In [ ]:
!python train.py --data data.yaml --cfg yolov5s.yaml --batch-size 8 --name Model --epochs 30

Streaming output truncated to the last 5000 lines.
      15/29      2.43G    0.01821    0.01599          0         35        640:   1% 2/164 [00:00<00:20,  7.77it/s]/content/drive/MyDrive/Colab Notebooks/Yolo_training/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      15/29      2.43G     0.0191    0.01476          0         28        640:   2% 3/164 [00:00<00:19,  8.16it/s]/content/drive/MyDrive/Colab Notebooks/Yolo_training/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      15/29      2.43G    0.01903    0.01509          0         39        640:   2% 4/164 [00:00<00:18,  8.43it/s]/content/drive/MyDrive/Colab Notebooks/Yolo_training/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. 

Training Yolo V5 Model

In [ ]:
!python export.py --weights runs/train/Models/weights/best.pt --include torchscript onnx

export: data=data/coco128.yaml, weights=['runs/train/Models/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
fatal: cannot change to '/content/drive/MyDrive/Colab': No such file or directory
YOLOv5 🚀 2025-5-26 Python-3.11.12 torch-2.6.0+cu124 CPU

Traceback (most recent call last):
  File "/content/drive/MyDrive/Colab Notebooks/Yolo_training/yolov5/export.py", line 1546, in <module>
    main(opt)
  File "/content/drive/MyDrive/Colab Notebooks/Yolo_training/yolov5/export.py", line 1541, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^